In [2]:
import cudf as cd
import cupy as cp
import pandas as pd
import numpy as np

In [3]:
dftrain=cd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [4]:
dftrain.info()

In [5]:
for i in dftrain:
    if dftrain[i].isnull().sum() > 100:
        dftrain=dftrain.drop(i,axis=1)
dftrain.info()

In [6]:
toprocess=[]
for i in dftrain:
    if dftrain[i].isnull().sum() > 0:
        toprocess.append(i)
toprocess

In [7]:
inttoprocess=dftrain[toprocess].select_dtypes(include=['float64']).columns
cattoprocess=dftrain[toprocess].select_dtypes(include=['object']).columns

In [8]:
for i in inttoprocess:
    dftrain[i]=dftrain[i].fillna(dftrain[i].mean())

In [9]:
dftrain=dftrain.dropna(subset=cattoprocess)
dftrain.info()

In [10]:
#No Null values in Training Data
toprocess1=[]
for i in dftrain:
    if dftrain[i].isnull().sum() > 0:
        toprocess1.append(i)
toprocess1

In [11]:
catdata=dftrain.select_dtypes(include=['object']).columns
catdata

In [12]:
#Encoding Categorical Data in Training Data
dftrain=cd.get_dummies(dftrain,columns=catdata)
dftrain.info()

In [13]:
dftrainx=dftrain.drop('SalePrice',axis=1)

In [14]:
from cuml.preprocessing import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(dftrainx,dftrain['SalePrice'],test_size=0.3,random_state=42)

In [15]:
#Scaling Features
from cuml.preprocessing import StandardScaler
ss=StandardScaler()
sdftrain=ss.fit_transform(xtrain)
tdftrain=ss.fit_transform(xtest)

In [16]:
#Dimensionality Reduction
from cuml.decomposition import PCA
pca=PCA(n_components=182)
pca.fit(sdftrain)
x=pca.transform(sdftrain)
pca.fit(sdftrain)
xt=pca.transform(tdftrain)
print(xt.shape)

In [17]:
from cuml import LinearRegression
lreig=LinearRegression(fit_intercept = True, normalize = False,algorithm = 'eig')
lreig.fit(x,ytrain)
lrsvd=LinearRegression(fit_intercept = True, normalize = False,algorithm = 'svd')
lrsvd.fit(x,ytrain)
lrsvdj=LinearRegression(fit_intercept = True, normalize = False,algorithm = 'svd-jacobi')
lrsvdj.fit(x,ytrain)
lrqr=LinearRegression(fit_intercept = True, normalize = False,algorithm = 'qr')
lrqr.fit(x,ytrain)
lrsqr=LinearRegression(fit_intercept = True, normalize = False,algorithm = 'svd-qr')
lrsqr.fit(x,ytrain)

In [18]:
#eig
ypredeig=lreig.predict(xt)
ypreddeig=ypredeig.astype(int)
ytestteig=ytest.astype(float)
#svd
ypredsvd=lrsvd.predict(xt)
ypreddsvd=ypredsvd.astype(int)
ytesttsvd=ytest.astype(float)
#svd-jacobi
ypredsvdj=lrsvdj.predict(xt)
ypreddsvdj=ypredsvdj.astype(int)
ytesttsvdj=ytest.astype(float)
#qr
ypredqr=lrqr.predict(xt)
ypreddqr=ypredqr.astype(int)
ytesttqr=ytest.astype(float)
#svd-qr
ypredsqr=lrsqr.predict(xt)
ypreddsqr=ypredsqr.astype(int)
ytesttsqr=ytest.astype(float)

In [19]:
from cuml.metrics.regression import mean_absolute_error,mean_squared_error,r2_score
print('Algorithm: \'eig\'')
print('MAE:',mean_absolute_error(ytest,ypreddeig))
print('MSE:',mean_squared_error(ytest,ypreddeig))
print('R2 Score:',r2_score(ytestteig,ypredeig)*100,'\n')
print('Algorithm: \'svd\'')
print('MAE:',mean_absolute_error(ytest,ypreddsvd))
print('MSE:',mean_squared_error(ytest,ypreddsvd))
print('R2 Score:',r2_score(ytesttsvd,ypredsvd)*100,'\n')
print('Algorithm: \'svd-jacobi\'')
print('MAE:',mean_absolute_error(ytest,ypreddsvdj))
print('MSE:',mean_squared_error(ytest,ypreddsvdj))
print('R2 Score:',r2_score(ytesttsvdj,ypredsvdj)*100,'\n')
print('Algorithm: \'qr\'')
print('MAE:',mean_absolute_error(ytest,ypreddqr))
print('MSE:',mean_squared_error(ytest,ypreddqr))
print('R2 Score:',r2_score(ytesttqr,ypredqr)*100,'\n')
print('Algorithm: \'svd-qr\'')
print('MAE:',mean_absolute_error(ytest,ypreddsqr))
print('MSE:',mean_squared_error(ytest,ypreddsqr))
print('R2 Score:',r2_score(ytesttsqr,ypredsqr)*100)